## Initial imports

In [ ]:
import neptune
import neptune.new as neptune
import os

token = os.getenv('NEPTUNE_API_TOKEN')
proj = 'joinemio/test'
run = neptune.init(project=proj,
                   api_token=token)
import time
import gym
import torch
import random
from collections import defaultdict, deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque

import numpy as np
import copy
from pprint import pprint
import math
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
import gc
import gym_joinemio
from gym_joinemio.envs.player import RandomPlayer
from gym_joinemio.envs.connect_four_env import Reward

Neptune prep

In [ ]:
env = gym.make('joinemio-v0')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

## Replay memory

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size) # TODO: Batch size delete

    def __len__(self):
        return len(self.memory)

## Training parameters: 
Random vs Random. Deep Q-Learning. Params:

- `n_episodes` (int): maximum number of training epsiodes
- `max_t` (int): maximum number of timesteps per episode _// Not used, because these episodes don't take too long and we like when game's are finished_
- `eps_start` (float): starting value of epsilon, for epsilon-greedy action selection
- `eps_end` (float): minimum value of epsilon 
- `eps_decay` (float): mutiplicative factor (per episode) for decreasing epsilon

In [ ]:
params = {
	"EPISODES" 					: 10,
	"EPS_START" 				: 1.0,
	"EPS_END" 					: 0.01,
	"LEARNING_RATE" 			: 1e-4,
	"EPS_DECAY" 				: 0.996,
	"EPS_DECAY_LAST_FRAME" 		: 10**5,
	"BUFFER_SIZE"				: 300,
	"FRAMES_MIN" 				: 15000,
	"MEAN_REWARD_BOUND" 		: 0.80,
	"SYNC_TARGET_FRAMES" 		: 1000,
	"REPLAY_START_SIZE" 		: 10000
}

run["parameters"] = params

EPISODES = params["EPISODES"]
EPS_START = params["EPS_START"]
EPS_END = params["EPS_END"]
LEARNING_RATE = params["LEARNING_RATE"]
EPS_DECAY = params["EPS_DECAY"]
EPS_DECAY_LAST_FRAME = params["EPS_DECAY_LAST_FRAME"] # 10**5

BUFFER_SIZE = params["BUFFER_SIZE"]
FRAMES_MIN = params["FRAMES_MIN"]
MEAN_REWARD_BOUND = params["MEAN_REWARD_BOUND"]
N_FOR_MEAN_REWARD = BUFFER_SIZE

SYNC_TARGET_FRAMES = params["SYNC_TARGET_FRAMES"]
REPLAY_START_SIZE = params["REPLAY_START_SIZE"]

## NeuralNetwork

In [ ]:
from gym_joinemio.envs.board import Board, GameState


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.main_layers = nn.Sequential(
            nn.Linear(6*7, 6*7),
            nn.ReLU(),
            nn.Linear(6*7, 6*7),
            nn.ReLU(),
            nn.Linear(6*7, 6*7),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(6*7,7)

    def forward(self, board_flatten_state):
        weights = self.output_layer(self.main_layers(board_flatten_state)) # weights = nn.relU(self.output_layer(self.main_layers(board_flatten_state)))
        return weights

## AIPlayer

In [ ]:
class AIPlayer:
    @staticmethod
    def possible_moves(board_state):
        available_cols = []
        for i in range(len(board_state[0])):
            if board_state[0][i] == 0:
                available_cols.append(i)
        return available_cols

    def __init__(self, env, replay_memory, net):
        self.net = net
        self.env = env
        self.env.opponent_action_set(RandomPlayer.get_action)
        self.replay_memory = replay_memory
        self._reset()

    def _reset(self):
        self.env = gym.make('joinemio-v0')
        self.state =  self.env.reset()
        self.env.opponent_action_set(RandomPlayer.get_action)#self.get_action)
        self.env.game = gym_joinemio.envs.board.Game()
        self.total_reward = 0.0

    def get_action(self, board_state):
        weigths = self.net.forward(torch.flatten(torch.FloatTensor(board_state.astype(float)).type(torch.FloatTensor)))
        pos_nums = self.possible_moves(board_state)
        max_num = 0
        for col in pos_nums:
            if weigths[max_num] < weigths[int(col)]:
                max_num = int(col)
        return max_num

    def get_net_action(self, grid):
        state_a = np.array(self.state, copy=False, dtype=np.uint8)
        state_v = torch.flatten(torch.FloatTensor(state_a).to(device))
        q_vals_v = self.net(state_v)
        # TODO filter
        for i in range(Board.columns):
            if i not in self.possible_moves(grid):
                q_vals_v[i] = -100
        _, act_v = torch.max(q_vals_v, dim=0)
        action = int(act_v.item())
        return action

    def play_step(self, net, epsilon, device):
        done_reward = None
        final_reward = None
        ## print("*AI PLAYER PLAYED*")
        state_a = np.array(self.state, copy=False, dtype=np.uint8)
        state_v = torch.flatten(torch.FloatTensor(state_a).to(device))
        q_vals_v = net(state_v)

        grid = self.env.game.board.grid
        for i in range(Board.columns):
            if i not in self.possible_moves(grid):
                q_vals_v[i] = -1
        _, act_v = torch.max(q_vals_v, dim=0)
        action = int(act_v.item())  # TODO: Later check if it returns correct action int range.

        new_state, reward, is_done, _ = self.env.step(action)
        final_reward = reward
        self.total_reward += reward.value

        self.replay_memory.push(self.state, action, new_state, reward)
        self.state = new_state

        if is_done == GameState.finished:
            done_reward = self.total_reward
            ## print(f"WINNER: player {self.env.game.winner} | Final reward reading: {final_reward}")
            ## print(self.env.game.board.grid)
            self._reset()
            return (done_reward, final_reward)
        else: return (done_reward, None)

    def train(self, memory_buffer, batch_size):
        return 0


## Training loop:

In [ ]:
from gym_joinemio.envs.game_window import GameWindow
from pyglet import clock
from pyglet import app

def play_with_ai(agent):
    window = GameWindow(agent) #(agent.get_net_action, agent.env.game)
    clock.schedule_interval(window.update, 1000) # GameWindow.refresh_rate)
    app.run()

def our_main():
    env = gym.make('joinemio-v0')
    net = NeuralNetwork().to(device)
    tgt_net = NeuralNetwork().to(device)
    print(net)
    buffer = ReplayMemory(BUFFER_SIZE)
    global agent 
    agent = AIPlayer(env, buffer, net) # TODO: params? Whatever agent is...
    epsilon = EPS_START
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    best_mean_reward = None
    last_total_frames = 0
    while True:
        frame_idx += 1
        epsilon = max(EPS_END, EPS_START - frame_idx / EPS_DECAY_LAST_FRAME)
        reward, final_reward = agent.play_step(net, epsilon, device=device)
        if (final_reward is not None):
            frames_this_game = frame_idx - last_total_frames
            last_total_frames = frame_idx
            ## print("Reward is not None (Game Finished)")
            total_rewards.append(reward)
            ts_frame = frame_idx
            ts = time.time()
            mean_reward = np.mean(total_rewards[-N_FOR_MEAN_REWARD:])
            ## print(f'Total Steps Played: {frame_idx}, steps this game {frames_this_game}, Games done: {len(total_rewards)}, mean reward: {mean_reward}, eps: {epsilon}, final reward: {final_reward}')
            ## print("")          # TODO: # Neptun logging (write epsilon, speed, reward_100, reward)
            run["eps"].log(epsilon)
            run["games_done"].log(len(total_rewards))
            run["mean_reward"].log(mean_reward)
            
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), "joinemio-best.dat") # TODO: Extract
                if best_mean_reward is not None:
                    print(f"Best mean reward updated {best_mean_reward}->{mean_reward}; model saved")
                best_mean_reward = mean_reward

            condition = mean_reward > MEAN_REWARD_BOUND and frame_idx > FRAMES_MIN
            if condition: 
                ## print(f"Solved in {frame_idx} frames")
                break
        else: pass ## print("REWARD NONE")
        
        if len(buffer) < REPLAY_START_SIZE:
            continue

        if frame_idx % SYNC_TARGET_FRAMES == 0:
            tgt_net.load_state_dict(net.state_dict())

        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        # loss_t = calc_loss(batch, net, tgt_net, device=device) # TODO: calc_loss method
        optimizer.step()

    # play_with_ai(agent)

our_main()

In [ ]:
run.stop()

In [ ]:
play_with_ai(agent)